# Création de prompts pour le nommage et la description des clusters

In [2]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "fr_CA.UTF-8/fr_CA.UTF-8/fr_CA.UTF-8/C/fr_CA.UTF-8/C"

In [3]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_hommes     <- mean(cluster_data$ses_genderMale, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$ses_immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$ses_languageEnglish, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$ses_languageFrench, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethnicityWhite, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_sexOrientationHetero, na.rm = TRUE) * 100
  educ_mean_BHS       <- mean(cluster_data$ses_educBHS, na.rm = TRUE) * 100
  educ_mean_PostHS       <- mean(cluster_data$ses_educPostHS, na.rm = TRUE) * 100
  educ_mean_Univ       <- mean(cluster_data$ses_educUniv, na.rm = TRUE) * 100
  age_mean        <- mean(cluster_data$ses_age, na.rm = TRUE) * 80
  revenu_moyen_Low    <- mean(cluster_data$ses_incomeLow, na.rm = TRUE) * 100
  revenu_moyen_Mid    <- mean(cluster_data$ses_incomeMid, na.rm = TRUE) * 100
  revenu_moyen_High    <- mean(cluster_data$ses_incomeHigh, na.rm = TRUE) * 100
  perc_pecheurs        <- mean(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE) * 100
  perc_chasseurs        <- mean(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE) * 100
  perc_motorized        <- mean(cluster_data$lifestyle_motorizedActFreq_bin, na.rm = TRUE) * 100
  perc_unmotorized   <- mean(cluster_data$lifestyle_unmotorizedActFreq_bin, na.rm = TRUE) * 100
  perc_tofu   <- mean(cluster_data$lifestyle_fridgeTofuTempeh, na.rm = TRUE) * 100
  perc_tattoos  <- mean(cluster_data$lifestyle_hasTattoos, na.rm = TRUE) * 100

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
  "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona ", cluster_value, " :\n\n",
  
  "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "green")) {
    paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
  } else {
    "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
  },
  
  "\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
  if (any(cluster_vars$color == "red")) {
    paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
  } else {
    "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
  },
  
  "\n\nCe cluster est composé de :\n", 
  "Hommes: ", round(perc_hommes, 1), "%, ",
  "Immigrants: ", round(perc_immigrants, 1), "%\n",
  "Âge: ", round(age_mean, 1), " ans\n",
  "Revenus:\n",
  "  Revenu Bas: ", round(revenu_moyen_Low, 1), " %\n",
  "  Revenu Mid: ", round(revenu_moyen_Mid, 1), " %\n",
  "  Revenu High: ", round(revenu_moyen_High, 1), " %\n",
  "Langues:\n",
  "  En: ", round(perc_langEn, 1), "%, ",
  "Fr: ", round(perc_langFr, 1), "%, ",
  "Autres: ", round(perc_ses_languageOther, 1), "%\n",
  "EducBHS: ", round(educ_mean_BHS, 1), "%\n",
  "EducPostHS: ", round(educ_mean_PostHS, 1), "%\n",
  "EducUniv: ", round(educ_mean_Univ, 1), "%\n",
  "Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
  "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n",
  "Pêcheurs: ", round(perc_pecheurs, 1), "%\n",
  "Chasseurs: ", round(perc_chasseurs, 1), "%\n",
  "Font du sport motorisé: ", round(perc_motorized, 1), "%\n",
  "Font du sport non-motorisé: ", round(perc_unmotorized, 1), "%\n",
  "Ont du tofu dans leur frigo: ", round(perc_tofu, 1), "%\n",
  "Tattoos: ", round(perc_tattoos, 1), "%\n"
))

  prompts[[cluster_value]] <- prompt_description
}


Warning message in mean.default(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_motorizedActFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_unmotorizedActFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_fridgeTofuTempeh, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goFishingFreq_bin, na.rm = TRUE):
“l'argument n'est ni numérique, ni logique : renvoi de NA”
Warning message in mean.default(cluster_data$lifestyle_goHuntingFreq_bin, na.rm = TRUE):
“l'argument n'es

In [4]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceCPC\", \"ses_regionWest\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_incomeLow\", \"ses_languageFrench\", \"ses_regionOntario\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 63.5%, Immigrants: 16.3%\nÂge: 45.8 ans\nRevenus:\n  Revenu Bas: 6.7 %\n  Revenu Mid: 78.8 %\n  Revenu High: 14.4 %\nLangues:\n  En: 97.1%, Fr: 0%, Autres: 2.9%\nEducBHS: 25%\nEducPostHS: 35.6%\nEducUniv: 39.4%\nEthnie (Blanc): 77.9%\nOrientation (Hétéro): 94.2%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 25%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceCPC\", \"ses_educUniv\", \"ses_immigrant\", \"ses_incomeHigh\", \"ses_regionOntario\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_educBHS\", \"ses_languageFrench\", \"ses_regionQuebec\", \"ses_regionWest\")\n\nCe cluster est composé de :\nHommes: 51.2%, Immigrants: 25.2%\nÂge: 46.3 ans\nRevenus:\n  Revenu Bas: 12.6 %\n  Revenu Mid: 60.6 %\n  Revenu High: 26.8 %\nLangues:\n  En: 87.4%, Fr: 1.6%, Autres: 11%\nEducBHS: 11%\nEducPostHS: 29.1%\nEducUniv: 59.8%\nEthnie (Blanc): 71.7%\nOrientation (Hétéro): 94.5%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 25.2%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 3 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_solidity\", \"dv_turnout\", \"dv_voteChoiceBQ\", \"ses_age\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceCPC\", \"dv_voteChoiceLPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_immigrant\", \"ses_incomeHigh\", \"ses_regionOntario\", \"ses_regionWest\", \"ses_urban\")\n\nCe cluster est composé de :\nHommes: 59.8%, Immigrants: 4.1%\nÂge: 64.9 ans\nRevenus:\n  Revenu Bas: 12.3 %\n  Revenu Mid: 82 %\n  Revenu High: 5.7 %\nLangues:\n  En: 3.3%, Fr: 96.7%, Autres: 0%\nEducBHS: 17.2%\nEducPostHS: 41%\nEducUniv: 41.8%\nEthnie (Blanc): 95.9%\nOrientation (Hétéro): 91%\nPêcheurs: NA%\nChasseurs: NA%\nFont du sport motorisé: NA%\nFont du sport non-motorisé: NA%\nOnt du tofu dans leur frigo: NA%\nTattoos: 22.1%\n"

$`4`
[1] "Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 4 :\n\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"dv_voteChoiceLPC\", \"ses_educUniv\", \"ses_regionOntario\", \"ses_urban\")\n\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"dv_voteChoiceBQ\", \"dv_voteChoiceCPC\", \"dv_voteChoiceNDP\", \"dv_voteChoiceOther\", \"ses_languageFrench\", \"ses_regionQuebec\")\n\nCe cluster est composé de :\nHommes: 48.3%, Immigrants: 19.7%\nÂge: 48.7 ans\nRevenus:\n  Revenu Bas: 11.6 %\n  Revenu Mid: 76.2 %\n  Revenu High: 12.2 %\nLangues:\n  En: 93.9%, Fr: 1.4%, Autres: 4.8%\nEducBHS: 17%\nEducPostHS: 27.2%\nEducUniv: 55.8%\nEthn

In [5]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [11]:
reponses_openai

$`1`
[1] "Pour ce persona, je propose le prénom \"Brad\". \n\n**Brad** est un homme de 46 ans vivant dans l'ouest du Canada. Il est anglophone et fait partie d'une famille établie dans la région depuis plusieurs générations, bien qu'il ait des amis immigrants et apprécie la diversité culturelle. Son revenu est dans la tranche moyenne, ce qui lui permet de mener une vie confortable sans excès. Bien éduqué, Brad a atteint un niveau universitaire et travaille probablement dans un secteur qui valorise la solidité et la stabilité, telles que l'ingénierie ou les finances.\n\nIl s'intéresse à la politique et vote, en général, pour le Parti conservateur du Canada, partageant les valeurs de ce parti. Brad n'a pas d'affinité particulière pour les partis comme le Bloc québécois, le Parti libéral, ou le NPD. Il est assez ancré dans sa communauté et valorise les activités de plein air, même si les précisions sur ses loisirs ne sont pas disponibles.\n\nBien que Brad ait un ou plusieurs tatouages, il reste plutôt conventionnel dans son style de vie et ses préférences. Il est hétérosexuel, ce qui, combiné à son ancrage régional et professionnel, lui offre une vie relativement stable."

$`2`
[1] "Pour une personne vivant au Canada, avec les caractéristiques du persona 2, je propose le prénom \"Alexandre\". Ce prénom est courant au Canada et est utilisé dans les deux langues officielles du pays, bien qu'il ait une connotation plus francophone.\n\nDescription du persona 2 : \n\nAlexandre est un homme de 46 ans vivant en Ontario, au Canada. Il fait partie d'une élite socio-économique avec un haut niveau d'éducation, ayant terminé des études universitaires (59.8%). Il est immigrant (25.2%) et parle principalement anglais (87.4%). Ses revenus sont majoritairement élevés, avec une proportion significative dans la tranche de revenu élevé (26.8%).\n\nPolitiquement, Alexandre a tendance à voter pour le Parti conservateur du Canada (CPC), avec un taux de participation élevé aux élections. Il est peu susceptible de soutenir des partis comme le Bloc Québécois, le Parti libéral du Canada, le Nouveau Parti démocratique ou d'autres petits partis. \n\nIl s'identifie en grande partie comme blanc (71.7%) et hétérosexuel (94.5%). Bien qu'il ne soit pas spécifiquement mentionné comme pêcheur ou chasseur, Alexandre pourrait être intéressé par d'autres activités et loisirs, mais ceux-ci ne sont pas définis. De plus, il a un tatouage, soulignant peut-être un aspect de sa personnalité ou de son style de vie unique (25.2%). \n\nAlexandre est probablement bien intégré dans sa communauté en Ontario, avec une vie professionnelle stable et une forte implication dans les affaires locales, reflétant sa participation politique élevée et son niveau d'éducation."

$`3`
[1] "Pour une personne habitant au Canada, en particulier au Québec, avec un profil correspondant à ces caractéristiques, je propose le prénom \"Luc\".\n\n### Description pour Persona 3 : Luc\n\n**Nom :** Luc Tremblay  \n**Âge :** 65 ans  \n**Sexe :** Homme  \n**Origine :** Québécois, Blanc  \n**Langue :** Francophone  \n**Région :** Québec  \n**Statut d'immigration :** Non-immigrant  \n**Orientation sexuelle :** Hétérosexuel  \n**Niveau d'éducation :** Universitaire  \n**Revenu :** Moyen\n\n**Profil :**\n\nLuc est un homme francophone de 65 ans vivant au Québec, fier de sa culture québécoise et souvent engagé dans des discussions politiques. Il affiche une forte solidité dans ses opinions et a une tendance marquée à voter pour le Bloc Québécois, alignant ses intérêts avec ceux qui favorisent la souveraineté et les intérêts du Québec. Avec sa longue expérience de vie, il valorise la stabilité et la solidarité communautaire.\n\nBien qu'il ait une éducation universitaire, Luc vit avec un revenu moyen. Il n'est pas particulièrement intéressé par les hautes sphères de revenus et préfère la simplicité et le confort d'une vie bien ancrée dans sa région. Très peu intéressé par l'anglais, Luc parle principalement frança

In [7]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [8]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_solidity"", ""dv_turnout"", ""dv_voteChoiceCPC"", ""ses_regionWest"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""dv_voteChoiceLPC"", ""dv_voteChoiceNDP"", ""dv_voteChoiceOther"", ""ses_incomeLow"", ""ses_languageFrench"", ""ses_regionOntario"", ""ses_regionQuebec"") Ce cluster est composé de : Hommes: 63.5%, Immigrants: 16.3% Âge: 45.8 ans Revenus: Revenu Bas: 6.7 % Revenu Mid: 78.8 % Revenu High: 14.4 % Langues: En: 97.1%, Fr: 0%, Autres: 2.9% EducBHS: 25% EducPostHS: 35.6% EducUniv: 39.4% Ethnie (Blanc): 77.9% Orientation (Hétéro): 94.2% Pêcheurs: NA% Chasseurs: NA% Font du sport motorisé: NA% Font du sport non-motorisé: NA% Ont du tofu dans leur frigo: NA% Tattoos: 25%","Pour ce persona, je propose le prénom ""Brad"". **Brad** est un homme de 46 ans vivant dans l'ouest du Canada. Il est anglophone et fait partie d'une famille établie dans la région depuis plusieurs générations, bien qu'il ait des amis immigrants et apprécie la diversité culturelle. Son revenu est dans la tranche moyenne, ce qui lui permet de mener une vie confortable sans excès. Bien éduqué, Brad a atteint un niveau universitaire et travaille probablement dans un secteur qui valorise la solidité et la stabilité, telles que l'ingénierie ou les finances. Il s'intéresse à la politique et vote, en général, pour le Parti conservateur du Canada, partageant les valeurs de ce parti. Brad n'a pas d'affinité particulière pour les partis comme le Bloc québécois, le Parti libéral, ou le NPD. Il est assez ancré dans sa communauté et valorise les activités de plein air, même si les précisions sur ses loisirs ne sont pas disponibles. Bien que Brad ait un ou plusieurs tatouages, il reste plutôt conventionnel dans son style de vie et ses préférences. Il est hétérosexuel, ce qui, combiné à son ancrage régional et professionnel, lui offre une vie relativement stable."
2,"Propose un prénom pour une personne habitant au Canada et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""dv_solidity"", ""dv_turnout"", ""dv_voteChoiceCPC"", ""ses_educUniv"", ""ses_immigrant"", ""ses_incomeHigh"", ""ses_regionOntario"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""dv_voteChoiceBQ"", ""dv_voteChoiceLPC"", ""dv_voteChoiceNDP"", ""dv_voteChoiceOther"", ""ses_educBHS"", ""ses_languageFrench"", ""ses_regionQuebec"", ""ses_regionWest"") Ce cluster est composé de : Hommes: 51.2%, Immigrants: 25.2% Âge: 46.3 ans Revenus: Revenu Bas: 12.6 % Revenu Mid: 60.6 % Revenu High: 26.8 % Langues: En: 87.4%, Fr: 1.6%, Autres: 11% EducBHS: 11% EducPostHS: 29.1% EducUniv: 59.8% Ethnie (Blanc): 71.7% Orientation (Hétéro): 94.5% Pêcheurs: NA% Chasseurs: NA% Font du sport motorisé: NA% Font du sport non-motorisé: NA% Ont du tofu dans leur frigo: NA% Tattoos: 25.2%","Pour une personne vivant au Canada, avec les caractéristiques du persona 2, je propose le prénom ""Alexandre"". Ce prénom est courant au Canada et est utilisé dans les deux langues officielles du pays, bien qu'il ait une connotation plus francophone. Description du persona 2 : Alexandre est un homme de 46 ans vivant en Ontario, au Canada. Il fait partie d'une élite socio-économique avec un haut niveau d'éducation, ayant terminé des études universitaires (59.8%). Il est immigrant (25.2%) et parle principalement anglais (87.4%). Ses revenus sont majoritairement élevés, avec une proportion significative dans la tranche de revenu élevé (26.8%). Politiquement, Alexandre a tendance à voter pour le Parti conservateur du Canada (CPC), avec un taux de participation élevé

In [9]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [10]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")